In [16]:
import pandas as pd
import re
import numpy as np
from transformers import BertTokenizer
from transformers import RobertaTokenizer
from transformers import RobertaTokenizerFast

In [2]:
pred = pd.read_pickle('../input/tweet-sentiment-extraction/preds.pkl')

In [3]:
pred.head()

,textID,text,selected_text,sentiment,kfold,pred,raw_pred,start_char_prob,end_char_prob
0,f7fdea625a,i`m so bored i can barely even tweet. i have n...,bored,negative,0,bored,bored,"[0.3842555582523346, 2.708190368139185e-05, 0....","[0.00013593888434115797, 1.9451351818133844e-0..."
1,c19de2c75b,awwwwwwwwww thats jus...awwwww.did she get t...,awwwwwwwwww thats jus...awwwww.did she get to...,neutral,0,awwwwwwwwww thats jus...awwwww.did she get t...,awwwwwwwwww thats jus...awwwww.did she get t...,"[1.9253204300184734e-05, 0.9850127696990967, 0...","[4.894992144954813e-08, 0.0006059082224965096,..."
2,0f963af18f,I did not twitt yesterday cause it was a very ...,I can not sleep,negative,0,I can not sleep,I can not sleep,"[0.09800223261117935, 2.3971338123374153e-06, ...","[0.00022162309323903173, 6.86740293076582e-07,..."
3,0583c78cc1,Congratulation`s to phil packer on completing ...,Congratulation`s,positive,0,Congratulation,Congratulation,"[0.8340740203857422, 0.8340740203857422, 0.834...","[0.33658167719841003, 0.33658167719841003, 0.3..."
4,1cdb444ea5,O`Charleys? Pretty good. Especially when its f...,Pretty good.,positive,0,Pretty good.,Pretty good.,"[0.06446736305952072, 5.577873162110336e-05, 0...","[2.3554735889774747e-05, 1.728861934680026e-05..."


In [4]:
pred['whole'] = pred.apply(lambda x: len(x['selected_text'])/len(x['text'])>0.95, axis=1)

In [5]:
pred.groupby('sentiment')['whole'].mean()

sentiment
negative    0.139828
neutral     0.871098
positive    0.112445
Name: whole, dtype: float64

In [6]:
pred.shape

(27480, 10)

In [7]:
def jaccard_list(l1, l2):
    a = set(l1)
    b = set(l2)
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [8]:
def jaccard_string(s1, s2):
    a = set(s1.lower().split())
    b = set(s2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [9]:
pred.head()

,textID,text,selected_text,sentiment,kfold,pred,raw_pred,start_char_prob,end_char_prob,whole
0,f7fdea625a,i`m so bored i can barely even tweet. i have n...,bored,negative,0,bored,bored,"[0.3842555582523346, 2.708190368139185e-05, 0....","[0.00013593888434115797, 1.9451351818133844e-0...",False
1,c19de2c75b,awwwwwwwwww thats jus...awwwww.did she get t...,awwwwwwwwww thats jus...awwwww.did she get to...,neutral,0,awwwwwwwwww thats jus...awwwww.did she get t...,awwwwwwwwww thats jus...awwwww.did she get t...,"[1.9253204300184734e-05, 0.9850127696990967, 0...","[4.894992144954813e-08, 0.0006059082224965096,...",True
2,0f963af18f,I did not twitt yesterday cause it was a very ...,I can not sleep,negative,0,I can not sleep,I can not sleep,"[0.09800223261117935, 2.3971338123374153e-06, ...","[0.00022162309323903173, 6.86740293076582e-07,...",False
3,0583c78cc1,Congratulation`s to phil packer on completing ...,Congratulation`s,positive,0,Congratulation,Congratulation,"[0.8340740203857422, 0.8340740203857422, 0.834...","[0.33658167719841003, 0.33658167719841003, 0.3...",False
4,1cdb444ea5,O`Charleys? Pretty good. Especially when its f...,Pretty good.,positive,0,Pretty good.,Pretty good.,"[0.06446736305952072, 5.577873162110336e-05, 0...","[2.3554735889774747e-05, 1.728861934680026e-05...",False


In [10]:
pred[pred['sentiment']=='neutral'].head(15)

,textID,text,selected_text,sentiment,kfold,pred,raw_pred,start_char_prob,end_char_prob,whole
1,c19de2c75b,awwwwwwwwww thats jus...awwwww.did she get t...,awwwwwwwwww thats jus...awwwww.did she get to...,neutral,0,awwwwwwwwww thats jus...awwwww.did she get t...,awwwwwwwwww thats jus...awwwww.did she get t...,"[1.9253204300184734e-05, 0.9850127696990967, 0...","[4.894992144954813e-08, 0.0006059082224965096,...",True
5,50e0c09bc1,yeah although probably not on BGT bugger,yeah although probably not on BGT bugger,neutral,0,yeah although probably not on BGT bugger,yeah although probably not on BGT bugger,"[1.705077193037141e-05, 0.9996707439422607, 0....","[2.4227350792216384e-08, 2.3303473426494747e-0...",True
9,dc0d1fc43d,"Bah DHCP server, why must you keep falling on ...","Bah DHCP server, why must you keep falling on ...",neutral,0,"Bah DHCP server, why must you keep falling on ...","Bah DHCP server, why must you keep falling on ...","[0.9966632723808289, 0.9966632723808289, 0.996...","[3.4550514101283625e-05, 3.4550514101283625e-0...",True
13,e6e359e97b,"I believe that is true, FML is quite the phen...","I believe that is true, FML is quite the pheno...",neutral,0,"I believe that is true, FML is quite the phen...","I believe that is true, FML is quite the phen...","[9.736543870531023e-05, 0.9693472385406494, 8....","[6.984529932196892e-08, 7.207714133983245e-06,...",True
16,a4439217cb,I`m making a logo on photoshop & it didn`t wor...,I`m making a logo on photoshop & it didn`t wor...,neutral,0,I`m making a logo on photoshop & it didn`t wor...,I`m making a logo on photoshop & it didn`t wor...,"[0.9742904901504517, 1.3246437447378412e-05, 0...","[4.5215419959276915e-06, 5.534614615498867e-07...",True
18,62ac88ef56,"math is all that is left for the day, then the...","math is all that is left for the day, then the...",neutral,0,"math is all that is left for the day, then the...","math is all that is left for the day, then the...","[0.9993453621864319, 0.9993453621864319, 0.999...","[9.34509534999961e-06, 9.34509534999961e-06, 9...",True
19,d02b3284fd,*[ It`s time for TSCC! ],*[ It`s time for TSCC! ],neutral,0,*[ It`s time for TSCC! ],*[ It`s time for TSCC! ],"[0.8767515420913696, 0.8767515420913696, 7.305...","[0.0005558537086471915, 0.0005558537086471915,...",True
24,827e3beeb4,No I haven`t made my page yet. Had to work,No I haven`t made my page yet. Had to work,neutral,0,No I haven`t made my page yet. Had to work,No I haven`t made my page yet. Had to work,"[3.58248726115562e-05, 0.9994944334030151, 0.9...","[1.850335706876649e-08, 8.657701982883736e-06,...",True
26,07cc511566,he won`t reply,he won`t reply,neutral,0,he won`t reply,he won`t reply,"[0.00010110034781973809, 0.9996463060379028, 0...","[2.009790705415071e-06, 0.00025881349574774504...",False
27,99d16017ae,"yï¿½n tï¿½m, sang n?m s? th?y **** m?c Tr?n t...","yï¿½n tï¿½m, sang n?m s? th?y **** m?c Tr?n ti...",neutral,0,"yï¿½n tï¿½m, sang n?m s? th?y **** m?c Tr?n t...","yï¿½n tï¿½m, sang n?m s? th?y **** m?c Tr?n t...","[9.274935291614383e-05, 0.9708285331726074, 0....","[5.938537697147694e-07, 0.00011705719953170046...",True


In [11]:
print(pred.loc[16, 'text'])

I`m making a logo on photoshop & it didn`t work out..  hehe.. try again lah..


In [12]:
pred['jaccard'] = pred.apply(lambda x: jaccard_string(x['selected_text'], x['pred']), axis=1)

In [15]:
old_data = pd.read_csv('../input/tweet-sentiment-extraction/tweet_dataset.csv')
old_data.drop_duplicates(subset=['text'], inplace=True)

In [17]:
pattern = re.compile(r"@")
old_data['at_num'] = old_data['old_text'].apply(lambda x: len(pattern.findall(x)))

temp = pred.merge(old_data[['text','at_num']], on='text', how='left')
temp['at_num'] = temp['at_num'].apply(lambda x: min(x, 5))
pred['at_num'] = temp['at_num'].tolist()

In [18]:
pred[pred['sentiment']!='neutral'].groupby('at_num')['jaccard'].mean()

at_num
0    0.552064
1    0.559759
2    0.548578
3    0.468800
4    0.541497
5    0.502072
Name: jaccard, dtype: float64

In [19]:
pred['j2'] = pred.apply(lambda x: jaccard_string(x['selected_text'], x['text']), axis=1)

In [20]:
pred[pred['sentiment']!='neutral'].groupby('at_num')['j2'].mean()

at_num
0    0.332926
1    0.321344
2    0.256996
3    0.209169
4    0.213381
5    0.231660
Name: j2, dtype: float64

In [13]:
pred[pred.whole&(pred['sentiment']!='neutral')].tail()

,textID,text,selected_text,sentiment,kfold,pred,raw_pred,start_char_prob,end_char_prob,whole,jaccard
27420,2ee537aba8,Can`t wait to go to bed,Can`t wait to go to bed,positive,4,Can`t wait to go to bed,Can`t wait to go to bed,"[0.9953102469444275, 0.9953102469444275, 0.995...","[0.001950045465491712, 0.001950045465491712, 0...",True,1.000000
27435,e51529e319,****. My friend`s in such a state and I don`t ...,****. My friend`s in such a state and I don`t ...,negative,4,****.,****.,"[0.6364249587059021, 0.6364249587059021, 0.636...","[0.004135539289563894, 0.004135539289563894, 0...",True,0.043478
27446,0efadd6e63,"fair enough. actually, you dont have to give ...","fair enough. actually, you dont have to give m...",positive,4,"fair enough. actually, you dont have to give m...","fair enough. actually, you dont have to give m...","[8.267346856882796e-05, 0.541861355304718, 0.5...","[1.1919223652512301e-07, 0.17535899579524994, ...",True,1.000000
27456,f0183c1a1b,had a wonderful evening with those I love!,had a wonderful evening with those I love!,positive,4,wonderful evening with those I love!,wonderful evening with those I love!,"[0.41039279103279114, 0.41039279103279114, 0.4...","[0.0004923820961266756, 0.0004923820961266756,...",True,0.750000
27476,ab8d203651,thanks love ) btw happy mother`s day to your mom,thanks love ) btw happy mother`s day to your mom,positive,4,thanks love ) btw happy mother`s day to your mom,thanks love ) btw happy mother`s day to your mom,"[8.075985533650964e-05, 0.6283606886863708, 0....","[1.7634127402743616e-07, 0.21527116000652313, ...",True,1.000000


In [20]:
pred[pred['sentiment']!='neutral'].groupby('whole')['jaccard'].mean()

whole
False    0.574849
True     0.418186
Name: jaccard, dtype: float64

In [21]:
pred[pred['sentiment']!='neutral'].groupby('whole')['jaccard'].count()

whole
False    14310
True      2053
Name: jaccard, dtype: int64

In [17]:
pred.groupby('sentiment')['jaccard'].mean()

sentiment
negative    0.554439
neutral     0.976144
positive    0.555877
Name: jaccard, dtype: float64

In [16]:
for i in range(20):
    print(pred.sample(n=7000)['jaccard'].mean())

0.7260258378257805
0.7220286952546208
0.726751145076919
0.7351937837514402
0.7231000731632706
0.7279536093756269
0.7177591602683062
0.7272524382833072
0.732672319830251
0.7243853038905943
0.7285610194200824
0.7257553627232839
0.7267822058804645
0.717492472290953
0.7228176427741524
0.7243450473808926
0.7206148319774526
0.727037666933354
0.7242734681396491
0.724113986645593


In [ ]:
pred.groupby('sentiment')['jaccard'].mean()

In [ ]:
pred.groupby('sentiment')['jaccard'].mean()

In [ ]:
pred['delta_jaccard'] = pred['label_jaccard']-pred['jaccard']

In [86]:
pred[pred['label_jaccard']<0.5][['text','selected_text','sentiment','label2','pred','label_jaccard']].sample(n=10)

,text,selected_text,sentiment,label2,pred,label_jaccard
21540,Good Morning! It`s been a while since I`ve 'ta...,he awesome,positive,the awesome,Good Morning! It`s been a while since I`ve 'ta...,0.333333
9769,watching a youtube vid of sing paranoid live i...,so cut,positive,so cute,cute,0.333333
27314,that would mean me babe! but **** it my name i...,IN COO,positive,FREAKIN COOL,COOL!,0.000000
9260,_de_B You have to email her - she only gets wh...,Shame ab,negative,Shame about,Shame,0.333333
10153,"Had a crazy night, lost keys, walked home, mis...",I`m sa,negative,I`m sad,I`m sad,0.333333
23698,_babsi there`s a manual process... but it`s ta...,crappy ms,negative,crappy msft,crappy msft product,0.333333
7932,moving back home today. pro: obnoxiously close...,obnoxiously cl,negative,obnoxiously closer,obnoxiously,0.333333
4146,I`m Kinda sleepy.. I Was up too late texting a...,nice bo,positive,nice boy,nice,0.333333
4997,"oooh...I wish I could`ve gone there today, too...",as looking forwar,positive,was looking forward,wish,0.200000
23860,We`re idiots. Ok mostly I was skint but hell I...,re idiot,negative,re idiots,We`re idiots.,0.333333


In [87]:
print(pred[(pred['label_jaccard']>0.5)&(pred['jaccard']<0.5)].shape)
pred[(pred['label_jaccard']>0.5)&(pred['jaccard']<0.5)][['text','selected_text','sentiment','label2','pred','label_jaccard']].sample(n=10)

(7350, 21)


,text,selected_text,sentiment,label2,pred,label_jaccard
1575,Everyone on the eastcoast... the rain hates us...,the rain hates us,negative,the rain hates us,hates,1.0
10558,"Once again, I`m dealing with cancer in my dire...",Bugger Cancel sucks!,negative,Bugger Cancel sucks!,sucks!,1.0
857,Ouuchh! I hurt my index finger!! ahhhh,I hurt my index finger!! ahhhh,negative,I hurt my index finger!! ahhhh,hurt,1.0
26594,"Goodmorning twitter, oh my gosh, i woke up soo...","Goodmorning twitter, oh my gosh, i woke up soo...",positive,"Goodmorning twitter, oh my gosh, i woke up soo...",Happy,1.0
3366,good luck with you finals!!!,good luck with you finals!!!,positive,good luck with you finals!!!,good luck,1.0
14707,This will be the worst day ever....graduation,worst day ever...,negative,worst day ever...,worst,1.0
559,it`s not sooo noticable it depends on how you ...,isn`t perfect,negative,isn`t perfect,it`s not sooo noticable it depends on how you ...,1.0
3785,_Nicole wow you are lucky happy birthday!,wow you are lucky,positive,wow you are lucky,lucky happy birthday!,1.0
17465,...hey danny .. did u run already ??? hope you...,?? hope you have a good day ;) i love,positive,?? hope you have a good day ;) i love,i love you,1.0
6622,yea just fun dive..a bit scary sometimes but f...,.a bit scary sometimes but fun.,neutral,.a bit scary sometimes but fun.,yea just fun dive..a bit scary sometimes but f...,1.0


In [119]:
pred[(pred['label_jaccard']>0.5)&(pred['jaccard']<0.1)][['text','selected_text','sentiment','label2','pred','whole_pred']].sample(n=10)

,text,selected_text,sentiment,label2,pred,whole_pred
6773,omg me 2 !! Haha i was half an hour late 4 wor...,whoops,negative,whoops,omg me 2 !! Haha i was half an hour late,0.068344
24104,._.; Thanxxx ! Now with that message I just wa...,Thanxxx,positive,Thanxxx,Thanxxx ! Now with that message I just wanna l...,0.092494
15121,wishing I were somewhere else besides here!! D...,Don`t worry this won`t dampen my day and neith...,positive,Don`t worry this won`t dampen my day and neith...,wishing,0.160372
21717,"thats it, rub it in that ur seeing morrissey! ...","thats it, rub it in that ur seeing morrissey! ...",negative,"thats it, rub it in that ur seeing morrissey! ...",cry,0.045176
23605,Congrats Hughesy to you and Holly on the safe ...,Congrats Hughesy to you and Holly on the safe ...,positive,Congrats Hughesy to you and Holly on the safe ...,Congrats,0.091529
20280,boo! I must`ve deleted it from my phone! can`t...,can`t find it in my emails either!,negative,can`t find it in my emails either!,boo!,0.180488
8521,jungle book 2 is sooooo cute.. i have nothing ...,cute..,positive,cute..,cute.,0.052669
2605,nothings better then going out for chinese at ...,with ma fave cousins at,positive,with ma fave cousins at,nothings better,0.055982
21960,think you should catch up on your sleep befor ...,haha goodnight,positive,haha goodnight,goodnight<3,0.040349
8617,Chowder is the ****!!!!!!,****!,negative,****!,****!!!!!!,0.185898


In [89]:
print(pred.loc[27089]['text'])

mom just woke me u[p and i am so mad i was dreaming about shoes she whants me to go to the river it is so stupid


In [90]:
pred[pred['delta_jaccard']<0.2][['text','sentiment','selected_text','pred','jaccard','label_jaccard','whole_pred']].sample(n=10)

,text,sentiment,selected_text,pred,jaccard,label_jaccard,whole_pred
23632,"your the voice i hear inside my head, the reas...",neutral,"your the voice i hear inside my head, the reas...","your the voice i hear inside my head, the reas...",1.0,1.0,0.996764
6495,Reading through today`s paper along with some ...,neutral,Reading through today`s paper along with some ...,Reading through today`s paper along with some ...,1.0,1.0,0.985243
12568,"that is definitely an easier way of saying it,...",positive,easier,easier,1.0,1.0,0.178897
21215,is drawing a picture for her bubba,neutral,is drawing a picture for her bubba,is drawing a picture for her bubba,1.0,1.0,0.997497
2823,macbook dying. switching to iphone.,negative,dying.,dying.,1.0,1.0,0.069340
7961,I`m getting more and more followers... look ou...,neutral,I`m getting more and more followers... look ou...,I`m getting more and more followers... look ou...,1.0,1.0,0.996389
18410,Had to cancel subscriptions today to NZZ and E...,neutral,Had to cancel subscriptions today to NZZ and E...,Had to cancel subscriptions today to NZZ and E...,1.0,1.0,0.978450
26303,"back,had a great MAD monday meeting tomorrow",positive,great,great,1.0,1.0,0.116820
20911,- me and my mom`s cars were broken into. I fee...,negative,I feel violated.,I feel violated.,1.0,1.0,0.082451
199,Going to miss my roomie ... We will no longer ...,negative,miss,miss,1.0,1.0,0.068783


In [18]:
pred[pred['delta_jaccard']>0.8][['text','sentiment','selected_text','pred','jaccard','label_jaccard','score']].sample(n=10)

,text,sentiment,selected_text,pred,jaccard,label_jaccard,score
9720,I really hope you see my tweets. Sent you so m...,positive,please? *prays*,I really hope,0.000000,1.0,0.781391
20678,Date Like A Man So You Dont Get Played Like A ...,negative,Date Like A Man So You Dont Get Played Like A ...,****,0.100000,1.0,1.211964
25347,is not feelin well... i feel sooooo weak....i ...,negative,weak..,i feel sooooo weak....i hate bein sick in the ...,0.000000,1.0,0.733103
8721,Chem wasn`t better than physics. And now I`m s...,negative,Chem wasn`t better than physics. And now I`m s...,tired.,0.100000,1.0,0.886198
17204,"lol. When I went to buy my new laptop in Feb.,...",negative,sadfaced,sadfaced.,0.000000,1.0,1.404858
10831,I am up and feeling pretty **** gud! I dont ev...,positive,I am up and feeling pretty **** gud! I dont ev...,nice feeling,0.133333,1.0,0.976072
1839,its Monday and all is good,positive,its Monday and all is good,good,0.166667,1.0,1.603872
11246,I can`t believe how tired I am right now... I ...,negative,Exhaaaausted,tired I am right now... I don`t know if I can ...,0.000000,1.0,0.736420
4988,sadly awake. wondering about contact info for ...,negative,sadly awake.,sadly awake. wondering about contact info for ...,0.086957,1.0,0.952517
18027,I don`t like the possibility of the left side ...,negative,don`t like,I don`t like the possibility of the left side ...,0.181818,1.0,0.864112


In [18]:
pred.loc[5121, 'score']

1.204892635345459

In [19]:
pred.loc[5121,'words']

['I',
 'have',
 'such',
 'fantastic',
 'friends',
 ',',
 'including',
 'several',
 'ones',
 'met',
 'through',
 'here',
 '!',
 'thanks',
 'for',
 'being',
 'in',
 'my',
 'life',
 '-',
 'you',
 'are',
 'such',
 'amazing',
 'people',
 '!']

In [22]:
pred.loc[5264,'start_pred']

array([9.81185138e-02, 1.41678040e-03, 4.93733585e-03, 1.77191396e-03,
       6.82277530e-02, 8.49719346e-03, 3.49782445e-02, 5.02023160e-01,
       4.99137794e-04, 9.88097279e-04, 2.13963143e-03, 1.15508868e-04,
       1.77697721e-03, 8.86982400e-03, 3.39606334e-03, 3.18561494e-03,
       7.11473497e-03, 2.19332110e-02, 1.46088466e-01, 7.16459937e-04,
       1.19606929e-03, 3.02357739e-03, 2.29963195e-03, 1.07984255e-04,
       7.18443841e-03, 1.12657312e-04, 8.25565308e-03, 4.41139378e-03,
       6.29264468e-05, 4.17986093e-03, 9.03637338e-05, 8.02920293e-03,
       7.13601694e-05, 6.63818559e-03, 1.92632055e-04, 2.70112883e-04,
       3.70642096e-02, 1.51517124e-05, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00], dtype=float32)

In [23]:
pred.loc[5264,'end_pred']

array([3.56165925e-04, 2.31539452e-04, 8.73471654e-05, 1.59801613e-03,
       3.96914460e-04, 1.68386148e-04, 2.09067672e-04, 5.45909703e-01,
       1.93442404e-02, 1.57455006e-03, 1.45811267e-04, 4.44929227e-02,
       4.84313816e-03, 5.02101087e-04, 4.03384533e-04, 1.88563732e-04,
       1.12367103e-04, 5.48380078e-04, 1.91551939e-01, 8.76957644e-03,
       1.06053101e-03, 8.19328893e-03, 1.50607055e-04, 2.42818426e-02,
       3.08785646e-04, 9.46327951e-03, 1.39125797e-03, 8.18730987e-05,
       2.96283583e-03, 1.36234943e-04, 2.57435534e-03, 1.20302160e-04,
       2.61232071e-03, 1.66190744e-04, 1.52182893e-03, 9.62494873e-03,
       1.13905616e-01, 9.78956268e-06, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00], dtype=float32)